In [ ]:
# We will be using numpy to generate example IQ sames for decoding, as well as generating out own IQ
# samples for encoding and transmitting
import numpy as np

# We'll use plotly for all our graphing needs
import plotly.express as px

In [ ]:
# We'll start by generating an FSK signal, we'll come to the G (gaussian pulse shaping)
# part of it later.

# First I need some data to modulate. I'll just pick some arbitrary bytes that will
# demontrate transitions well
data = np.array([0x1b, 0x39, 0x12, 0x30], dtype=np.uint8)

# let's now split up our data into our symbol size. You never mention what form of FSK
# you need. We could transmit 1 bit at a time, or 2 bits at a time, or 4, or really any
# number of bits. I will use 2 bits per symbol, since the process will generalize well
# to any number of bits at a time. This is called 4-FSK because there are 4 possible
# symbols.

# unpack the data into bits
bits = np.unpackbits(data)
print(f"{bits=}")

# re-pack the data two bits at a time to achieve our desired bits per symbol
symbols = (bits[::2] << 1) + bits[1::2]
print(f"{symbols=}")

In [ ]:
# Now that we have our data, we need to consider our RF. I'm going to arbitrarily choose
# a carrier frequency of 146.000 MHz (in the 2 meter amateur radio band).
carrier_freq = 146e6

# Now I need to make an array containing this carrier wave. In order to make this array
# the proper length, now I need to choose how many samples per symbol I want. This is
# directly proportional to the time spent transmitting each symbol. This also means
# that our sample rate needs to be known now. Most SDRs can achieve 2 Msps, so I will
# choose that.

# lets choose 1 millisecond per symbol, and calculate how many samples that is.
samples_per_symbol = 2e6 * 1e-3
print(f"{samples_per_symbol=}")

In [ ]:
# Now it's time to modulate our signal into RF. For 4-FSK, we need 4 frequencies spaced
# evenly, two above and two below the carrier. You can do this with a for loop, but it
# is MUCH faster to get clever with matrix math and linear algebra with numpy.
# At this time we need to know our signal bandwidth. Lets choose 5kHz as an arbitrary
# but reasonable, if not slightly larger than needed value.

# First normalize our symbols from zero to one, then we center it around zero. Then
# we multiply our normalized, shifted frequencies by our bandwidth. This results in
# a max distance from our minimum frequency to maximum frequency of 5 kHz. We have
# negative frequencies, but that's ok! In the world of RF negative frequencies not
# only exist, but are incredibly useful.
freqs = ((symbols / 3) - 0.5).repeat(samples_per_symbol) * 5e3
px.scatter(
    x=np.linspace(0, len(freqs), len(freqs)),
    y=freqs
)

In [ ]:
# we will need a time vector to generate our signal. It should go from zero to the end
# of our packet. We will cut off the last number since we want it to be the same length
# as our frequencies array
time = np.arange(0, len(freqs) / 2e6, 1/2e6)[:-1]

# Now we can generate our carrier wave. We will use euler's identity to create a
# complex carrier wave that represents our I and our Q signals at our needed frequencies.
fsk_waveform = np.exp(1j * 2 * np.pi * freqs * time)

# Now let's see what it looks like!
px.scatter(
    {
        "time": time,
        "I": fsk_waveform.real,
        "Q": fsk_waveform.imag,
    },
    x="time",
    y=["I", "Q"],
)
# Looking at the plot below, we can clearly see two different frequencies being used
# to modulate our data. But didn't we need four?
# Yes! We have negative frequencies in here! Notice how in some parts, the I signal
# (blue) is leading the Q signal (red). These represent positive frequencies. When
# the Q leads the I, we have imaginary frequencies! This gives us our four different
# frequencies

In [ ]:
# Finally, in order to turn our FSK signal into GFSK, we need to run our modulated waveform
# through a Gaussian filter, in order to smooth out our transitions between frequencies. We
# smooth them out in order to prevent spectral leakage, or in other words to prevent unwanted
# frequencies from polluting our RF spectrum.

# Filtering is overall pretty straight forward. First, we generate a filtering "kernel" that
# can be reused as much as we want. First we decide how many "taps" our filter will have, or
# in other words how wide our filter is. We also need to choose the standard deviation of our
# filter, which determines how sharp our cutoffs are. We will choose 200 taps and a stdev of 30,
# simply because it works fine in our case (by mainly visual inspection.)
n_taps = 200
stdev = 30

# first make an array of x values to be used in the gaussian equation:
taps = np.arange(-n_taps // 2, n_taps // 2)

# Then use those numbers in the gaussian equation to generate an array of y values
filter = np.exp(-np.power(taps, 2) / (2 * stdev**2)) / (np.sqrt(2 * np.pi) * stdev)

# let's see what it looks like!
px.scatter(
    x=taps,
    y=[filter],
)

In [ ]:
# Now we will apply this filter to our FSK waveform to make GFSK! We will use convolution,
# which is a very important, but sort of hard to understand concept. I would recommend
# reading up on it from other sources.
gfsk_waveform = np.convolve(fsk_waveform, filter, "same")

px.scatter(
    {
        "time": time,
        "I": gfsk_waveform.real,
        "Q": gfsk_waveform.imag,
    },
    x="time",
    y=["I", "Q"],
)

# See how the filter smoothed out the transitions between different frequencies? This
# "Pulse Shaping" as it's called helps out signal come out of our transmitter more cleanly!
# There are other ways that we can clean up our signal more as well, but this is fine for now.

In [ ]:
# Now our `waveform` array is completely ready to be transmitted! Send it off into SoapySDR
# and enjoy the fruits of your labor! The SDR will handle taking our low frequency
# (-2.5 kHz to 2.5 kHz) up to radio frequencies. I won't show that here, I'll make a different
# tutorial for that later. Plus, a Jupyter notebook isn't the best format to do such things in.

In [ ]:
# Now to decode our signal that we just made! There are two important stages for decoding
# that I'm going to gloss over here though. Entire thesis papers can be and have been
# written on this subject matter, so it's a little deep for an introduction:
# Packet detecting and synchronization. It suffices me to say right now that these things
# can be done with something as simple as power detection and correlation stages in the
# decoder, for now we'll move onto the FSK specifics.
#
# Why not GFSK specifics? Because on the demodulator side, there is no difference between
# FSK and GFSK! They are demodulated exactly the same.

In [ ]:
# There are a ton of ways that we can demodulate the waveform. Most have pros and cons,
# and this one is no exception. It's probably more computationally expensive than
# optimal, but it's pretty simple to understand and implement

# First I make a complex (I and Q) wave at each frequency that we are going to demodulate.
# In this case, -2.5 kHz, -888 Hz, 888 Hz, and 2.5 kHz. We are going to stack all three
# arrays into a matrix so we can do some quick matrix math
symbol_carriers = np.stack(
    [
        np.exp(1j * 2 * np.pi * freq * time)
        for freq in np.linspace(0.5, -0.5, 4) * 5e3 # NOTE
    ]
)
# NOTE: this is reversed order on purpose so that symbols line up properly. I'm too
# tired too figure out why this is mathematically, but there we go.

# Multiply each row of the matrix (each carrier) by our waveform to demodulate. This is
# elementwise matrix multiplication done on each row individually.
demod = symbol_carriers[:,] * gfsk_waveform

# Now let's pick a symbol and see what it looks like after this stage of demodulating
target_symbol = 0
px.scatter(
    {
        "time": time,
        "I": demod[target_symbol].real,
        "Q": demod[target_symbol].imag,
        "ref": freqs / 5e3
    },
    x="time",
    y=["I", "Q", "ref"],
)
# We see that for whichever target symbol we choose, only the selected symbol demodulates
# to an average I value of one, and and average Q of zero. Again, this is one of many ways
# to accomplish this, but this works well enough

# We'll discard the Q values as we finish our demodulation. Make no mistake though, it was used
# in the demodulation through it's multiplying with our carriers.
float_demod = demod.real


In [ ]:
# Now we just need to compute what we see in the plot above. We need to average a number
# of samples equal to the length of each symbol.

# Averaging is done by reshaping our array so that the indexing is as follows:
# demod[symbol_carrier_indices, symbol_indices, samples]
# We do the reshaping because it allows us to easily sum up all of the entries in the
# third index. Then, by dividing by the number of samples, we get an average value.
# it's up to us to decide how close a symbol needs to be in order to classify it as a
# match or miss. In this case, I'll classify anything above a 0.5 to be a match.
threshold = 0.5

# 4 carriers, x symbols transmitted, and `samples_per_symbol` number of samples in each symbol.
reshaped_demod = float_demod.reshape(
    (4, int(float_demod.shape[1] / samples_per_symbol), int(samples_per_symbol))
)

# sum and divide to get our average
averaged_demod = np.sum(reshaped_demod, axis=2) / samples_per_symbol

# get a boolean array containing our symbols, then make it a uint8 array for further processing
demod_symbols = (averaged_demod > 0.5).astype(np.uint8)

# multiply each row by it's binary representation, and condense all the rows. Note
# the chance for optimization here, since the first row is always going to be all zeros
demod_representations = np.sum(
    demod_symbols * np.array([[0], [1], [2], [3]]),
    axis=0
)
print(demod_representations)

In [ ]:
# Now we just repack our symbols into whole bytes!
demodulated_data = (
    (demod_representations[0::4] << 6)
    + (demod_representations[1::4] << 4)
    + (demod_representations[2::4] << 2)
    + (demod_representations[3::4])
)
demodulated_data

In [ ]:
# We have reconstructed our GFSK data into the same bytes that we modulated earlier!
# Do note that, again, there are many ways to both modulate and demodulate the same signal.
# This method is likely neither the easiest, nor the most optimized, but it is valid.
# 
# Also note that I am missing the very important demodulation stages of packet detecting
# and synchronization, but those are beasts all their own, and I have not solidified my
# own knowledge in performing them from scratch, so I'll leave them to your own research.